In [134]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split

In [125]:
df = pd.read_excel(r'../data/result_prepared_df.xlsx', index_col=0)
df.head()

,ЖК,index(),Регион,Населенный пункт,Группа компаний,ID проекта,Класс недвижимости,Статус,"Продано квартир, руб","Продано нежилых, шт",...,Месяц_Май 2022,Месяц_Март 2021,Месяц_Март 2022,Месяц_Ноябрь 2021,Месяц_Октябрь 2021,Месяц_Сентябрь 2021,Месяц_Февраль 2021,Месяц_Февраль 2022,Месяц_Январь 2021,Месяц_Январь 2022
0,Восход 40376,258,Приморский край,Артем,СЗВОСХОД,11228,-1.341444,-0.321288,-0.889197,-0.287707,...,-0.262202,-0.1675,-0.290803,-0.240275,-0.219108,-0.22154,-0.164399,-0.276767,-0.161245,-0.253605
1,Восход 40376,261,Приморский край,Артем,СЗВОСХОД,11228,-1.341444,-0.321288,-0.824694,-0.287707,...,-0.262202,-0.1675,-0.290803,4.161902,-0.219108,-0.22154,-0.164399,-0.276767,-0.161245,-0.253605
2,Восход 40376,262,Приморский край,Артем,СЗВОСХОД,11228,-1.341444,-0.321288,-0.849961,-0.287707,...,-0.262202,-0.1675,-0.290803,-0.240275,-0.219108,-0.22154,-0.164399,-0.276767,-0.161245,-0.253605
3,Восход 40376,263,Приморский край,Артем,СЗВОСХОД,11228,-1.341444,-0.321288,-0.810597,-0.287707,...,-0.262202,-0.1675,-0.290803,-0.240275,-0.219108,-0.22154,-0.164399,-0.276767,-0.161245,3.943144
4,Восход 40376,264,Приморский край,Артем,СЗВОСХОД,11228,-1.341444,-0.321288,-0.868588,-0.287707,...,-0.262202,-0.1675,-0.290803,-0.240275,-0.219108,-0.22154,-0.164399,3.613142,-0.161245,-0.253605


# Дерево решений

In [126]:
x = df.drop(['Таргет'], axis=1)
y = df['Таргет']



In [127]:
mae_list = []
clf = DecisionTreeRegressor(random_state=42)

for i in range(50):    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

    x_train_numeric = x_train.select_dtypes(include=[np.number])
    x_test_numeric = x_test.select_dtypes(include=[np.number])

    clf.fit(x_train_numeric, y_train)

    y_pred = clf.predict(x_test_numeric)

    mae_list.append(mean_absolute_error(y_test, y_pred))

np.mean(mae_list), np.std(mae_list)


(15599.231683735065, 998.4204695845951)

In [128]:
clf.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 42,
 'splitter': 'best'}

# Случайный лес

In [156]:
rf_mae_list = []
base_rf = RandomForestRegressor(random_state=42)

for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

    x_train_numeric = x_train.select_dtypes(include=[np.number])
    x_test_numeric = x_test.select_dtypes(include=[np.number])

    base_rf.fit(x_train_numeric, y_train)

    y_base_rf_pred = base_rf.predict(x_test_numeric)

    rf_mae_list.append(mean_absolute_error(y_test, y_base_rf_pred))

np.mean(rf_mae_list), np.std(rf_mae_list)


(11532.019600337404, 516.9994083849957)

In [154]:
param_grid = {
   'bootstrap': [True],
   'n_estimators': list(range(400, 601, 100)),
   'max_depth': [80, 100],
   'max_features': ['sqrt'],
   'min_samples_leaf': list(range(1, 3)),
}


# param_grid = {
#     'bootstrap': [True],
#     'max_depth': [80, 90, 100, None],
#     'max_features': ["sqrt", "log2", None],
#     'min_samples_leaf': [1, 3, 5],
#     'min_samples_split': [2, 3, 4],
#     'n_estimators': list(range(500, 701, 100))
# }

rf = RandomForestRegressor(random_state=42)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

x_train_numeric = x_train.select_dtypes(include=[np.number])
x_test_numeric = x_test.select_dtypes(include=[np.number])

grid_search_rf = GridSearchCV(
   estimator=rf,
   param_grid=param_grid,
   scoring='neg_mean_absolute_error',
   verbose=1,
   # n_jobs=-1
)

grid_search_rf.fit(x_train_numeric, y_train)

# лучшие параметры
best_params = grid_search_rf.best_params_
best_params


# mae = ~12000, но очень долго работает, около 10 минут

# Определяем пространство поиска
# space = {
#     'n_estimators': hp.choice('n_estimators', range(500, 701, 100)),
#     'max_features': hp.choice('max_features', ['sqrt']),
#     'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 4))
# }

# # Функция для минимизации
# def objective(params):
#     rf = RandomForestRegressor(random_state=42, **params)
#     score = cross_val_score(rf, x_train_numeric, y_train, scoring='neg_mean_absolute_error').mean()
#     return {'loss': -score, 'status': STATUS_OK}

# # Запускаем поиск
# trials = Trials()
# best = fmin(fn=objective,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=100,
#             trials=trials)

# # Выводим лучшие параметры
# best_params = {
#     'n_estimators': range(500, 701, 100)[best['n_estimators']],
#     'max_features': ['sqrt'][best['max_features']],
#     'min_samples_leaf': range(1, 3)[best['min_samples_leaf']]
# }
# best_params

# # Обучаем модель с лучшими параметрами
# rf_tuned = RandomForestRegressor(random_state=42, **best_params)
# rf_tuned.fit(x_train_numeric, y_train)

# # Делаем предсказания на тестовых данных
# preds = rf_tuned.predict(x_test_numeric)

# # Вычисляем среднюю абсолютную ошибку
# mae = mean_absolute_error(y_test, preds)
# mae

Fitting 5 folds for each of 12 candidates, totalling 60 fits


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 400}

In [155]:
rf_tuned_mae_list = []
rf_tuned = RandomForestRegressor(**best_params, random_state=42)

for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)

    x_train_numeric = x_train.select_dtypes(include=[np.number])
    x_test_numeric = x_test.select_dtypes(include=[np.number])

    rf_tuned.fit(x_train_numeric, y_train)

    y_rf_tuned_pred = rf_tuned.predict(x_test_numeric)

    rf_tuned_mae_list.append(mean_absolute_error(y_test, y_rf_tuned_pred))

np.mean(rf_tuned_mae_list), np.std(rf_tuned_mae_list)

(13036.686926312743, 671.740302327626)